# S3W9D3_VectorDB_Demo

## 1 理论知识讲解

### 1.1 为什么不能用`for`循环？

如果有100万哥文档，每次查询都计算Query向量和这100万个向量的余弦相似度（线性扫描$O(N)$），延迟会达到秒级甚至分钟级，这在生产环境中是不可接受的。

我们需要将复杂度降到$O(\log N)$

### 1.2 HNSW索引（Hierachucal Navigable Small World）

目前业界最主流的向量索引算法。

- **CS 原型**：结合了 Skip List (跳表) 和 Small World Graph (小世界网络)。
- **结构**：
    - 它构建了一个**多层图结构**。
    - **Top Layers (高速公路)**：节点稀疏，连接距离远。搜索时先在这里快速逼近目标区域。
    - **Bottom Layers (社区街道)**：节点稠密，连接所有邻居。搜索到了这里，进行精细的局部查找。
- **Trade-off**：HNSW 是典型的**“空间换时间”**。它需要消耗大量的内存来存储节点之间的“边”（图的连接关系），换取极快的搜索速度。

## 2 核心难点：Chroma与我们模型的适配（The Adapter Pattern）

**⚠️ 注意**：ChromaDB 默认会尝试下载它内置的英文模型（all-MiniLM-L6-v2）。
为了使用我们在 Day 2 封装好的 **BGE-Small-Zh**（且保证离线可用），我们需要写一个**适配器（Adapter）**。

### Step 1: 定义适配器

In [3]:
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings

# 引入我们在Day2写的单例类
import sys
import os
sys.path.append(os.path.abspath("../../"))

from src.rag.embedding import EmbeddingModel

# --- 核心：编写适配器 ---
# Chroma 要求 Embedding 函数必须继承自 EmbeddingFunction
class MyCustomEmbeddingFunction(EmbeddingFunction):
    def __init__(self):
        #以此调用我们昨天写的单例
        self.model = EmbeddingModel() 

    def __call__(self, input: Documents) -> Embeddings:
        # 转换逻辑：调用我们的模型进行编码
        return self.model.get_embeddings(input)

print("✅ Adapter definition complete.")

✅ Adapter definition complete.


### Step 2: 初始化 Chroma 并存入数据

In [4]:
# 1. 初始化客户端 (持久化到本地磁盘)
client = chromadb.PersistentClient(path="./chroma_db_data")

# 2. 创建或获取 Collection (类似于 SQL 中的 Table)
# 必须传入我们自定义的 embedding_function，否则它会去下载默认模型
collection = client.get_or_create_collection(
    name="legal_knowledge_base",
    embedding_function=MyCustomEmbeddingFunction() 
)

# 3. 准备数据 (模拟切分后的 Chunks)
documents = [
    "盗窃公私财物，数额较大的，或者多次盗窃、入户盗窃、携带凶器盗窃、扒窃的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金。",
    "诈骗公私财物，数额较大的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金。",
    "故意伤害他人身体的，处三年以下有期徒刑、拘役或者管制。"
]
metadatas = [
    {"source": "刑法", "article": "264", "topic": "theft"},
    {"source": "刑法", "article": "266", "topic": "fraud"},
    {"source": "刑法", "article": "234", "topic": "injury"}
]
ids = ["doc_1", "doc_2", "doc_3"]

# 4. 写入数据 (Add)
# Chroma 会自动调用 embedding_function 把 documents 变成向量存起来
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

print(f"✅ Indexed {collection.count()} documents.")

Initializing Embedding Model: BAAI/bge-small-zh-v1.5...
Attempting to load from local cache (Offline Mode)...
✅ Successfully loaded from local cache.
✅ Indexed 3 documents.


### Step 3: 语义检索 (Query)

In [5]:
# 5. 查询 (Query)
# 用户问了一个模糊的问题，没有提到"盗窃"这个词
query_text = "偷东西怎么判？"

results = collection.query(
    query_texts=[query_text],
    n_results=1 # Top-1
)

# 6. 解析结果
print(f"\n🔍 Query: {query_text}")
print("-" * 30)
# results 是一个字典，包含 ids, distances, metadatas, documents
best_doc = results['documents'][0][0]
best_meta = results['metadatas'][0][0]
distance = results['distances'][0][0] # 默认是 L2 距离，越小越相似

print(f"Match: {best_doc}")
print(f"Source: {best_meta}")
print(f"Distance: {distance:.4f}")


🔍 Query: 偷东西怎么判？
------------------------------
Match: 盗窃公私财物，数额较大的，或者多次盗窃、入户盗窃、携带凶器盗窃、扒窃的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金。
Source: {'source': '刑法', 'topic': 'theft', 'article': '264'}
Distance: 0.7714


## 3. 工程封装 (Engineering Encapsulation)

在 Notebook 跑通后，我们要将逻辑封装到 `src/rag/vector_db.py` 中，使其成为一个通用的组件。

**代码焦点**：`src/rag/vector_db.py`

```python
import chromadb
from typing import List, Dict, Any
from src.rag.embedding import EmbeddingModel

# 再次定义适配器 (为了保持 vector_db.py 的独立性)
class MyCustomEmbeddingFunction(chromadb.EmbeddingFunction):
    def __init__(self):
        self.model = EmbeddingModel()

    def __call__(self, input: List[str]) -> List[List[float]]:
        return self.model.get_embeddings(input)

class VectorDB:
    def __init__(self, collection_name: str = "knowledge_base", persist_path: str = "./chroma_db"):
        """
        初始化向量数据库
        """
        self.client = chromadb.PersistentClient(path=persist_path)
        self.embedding_fn = MyCustomEmbeddingFunction()
        self.collection = self.client.get_or_create_collection(
            name=collection_name,
            embedding_function=self.embedding_fn,
            # HNSW 参数通常默认即可，如果有性能需求可在此调整 metadata={"hnsw:space": "cosine"}
            metadata={"hnsw:space": "cosine"} 
        )

    def add_documents(self, texts: List[str], metadatas: List[Dict[str, Any]], ids: List[str] = None):
        """
        批量添加文档
        """
        if ids is None:
            # 如果没给 ID，就用哈希生成，或者简单的 index
            ids = [str(hash(text)) for text in texts]
            
        self.collection.add(
            documents=texts,
            metadatas=metadatas,
            ids=ids
        )
        print(f"Successfully added {len(texts)} documents to collection.")

    def search(self, query: str, top_k: int = 3) -> List[Dict]:
        """
        语义搜索
        """
        results = self.collection.query(
            query_texts=[query],
            n_results=top_k
        )
        
        # 整理 Chroma 的返回格式，使其更易读
        structured_results = []
        if results['documents']:
            for i in range(len(results['documents'][0])):
                item = {
                    "text": results['documents'][0][i],
                    "metadata": results['metadatas'][0][i],
                    "distance": results['distances'][0][i],
                    "id": results['ids'][0][i]
                }
                structured_results.append(item)
        
        return structured_results

# 测试代码
if __name__ == "__main__":
    db = VectorDB(persist_path="./test_db")
    db.add_documents(
        texts=["测试文档一", "测试文档二"],
        metadatas=[{"id": 1}, {"id": 2}],
        ids=["1", "2"]
    )
    res = db.search("测试")
    print(res)

```

## 4. 模拟面试 (Mock Interview)

**面试官**：*“请简述 HNSW 索引的基本原理？在 RAG 系统中，如果我希望最近日期的文档优先被检索到，该怎么做？”*

**参考回答范式**：

> 1. **HNSW 原理**：它是基于小世界网络理论的图索引。你可以把它想象成**多层跳表**。最上层是稀疏的节点，用于快速的大步跳跃，锁定大致区域；越往下层节点越密集，用于精确查找最近邻。它通过牺牲内存（存储大量的边）来换取  的极快搜索速度。
> 2. **时间优先策略 (Hybrid Search)**：
>   * 单纯的向量搜索只看语义相似度，不看时间。
>   * 在 ChromaDB 或 Milvus 中，我们可以在 `query` 时加入 **Metadata Filtering (元数据过滤)**。
>   * 例如：`collection.query(..., where={"date": {"$gt": "2024-01-01"}})`。
>   * 或者，可以在拿到检索结果后，进行**重排序 (Re-ranking)**，通过加权公式让新文档的分数更高。
> 
> 
> 
> 


## 5. 今日 LeetCode 练习

* **题目**：[21. Merge Two Sorted Lists (合并两个有序链表)](https://leetcode.com/problems/merge-two-sorted-lists/)
* **关联 RAG**：这其实是 RAG 中 **"Reciprocal Rank Fusion (RRF)"** 算法的微观基础。当我们同时从关键字搜索（BM25）和向量搜索（Embedding）拿到两个有序列表时，我们需要将它们合并。
* **思路**：
* **递归法**：代码极简，但容易爆栈。
* **迭代法**：引入一个 `dummy` 节点（哨兵节点），比较 `l1.val` 和 `l2.val`，谁小就把谁接在后面。这是工程中最稳健的写法。



请在你的练习本中完成。



## 6. 今日任务总结

* [ ] 成功安装 `chromadb`。
* [ ] 编写了 `MyCustomEmbeddingFunction` 适配器，成功让 Chroma 用上了我们在 Day 2 写的本地模型。
* [ ] 封装了 `VectorDB` 类，支持 `add_documents` 和 `search`。
* [ ] 完成了 LeetCode 21。

**准备好代码封存了吗？**
执行完 Git 提交后，我们的 RAG 系统现在已经有了**大脑 (LLM)** 和 **长时记忆 (VectorDB)**。明天（Day 4），我们将编写一个 **ETL 脚本**，把真正的法律文档灌入数据库，让它真正跑起来。